# Homework 3 - Improving the Pipeline 
# Part 2: Analysis

CAPP 30235 Machine Learning for Public Policy

Jonathan Tan

May 2, 2019

## 0. Setup

In [65]:
# Setup autoreload
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [66]:
# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Import pipeline library, hardcoded config file values
import pipeline_library as library
import pipeline_config as config

## 1. Read data

In [67]:
df = library.read_data(config.DATA_PATH)
print(df.shape)
df.head()

(124976, 26)


,projectid,teacher_acctid,schoolid,school_ncesid,school_latitude,school_longitude,school_city,school_state,school_metro,school_district,...,secondary_focus_subject,secondary_focus_area,resource_type,poverty_level,grade_level,total_price_including_optional_support,students_reached,eligible_double_your_impact_match,date_posted,datefullyfunded
0,00001ccc0e81598c4bd86bacb94d7acb,96963218e74e10c3764a5cfb153e6fea,9f3f9f2c2da7edda5648ccd10554ed8c,1.709930e+11,41.807654,-87.673257,Chicago,IL,urban,Pershing Elem Network,...,Visual Arts,Music & The Arts,Supplies,highest poverty,Grades PreK-2,1498.61,31.0,f,4/14/13,5/2/13
1,0000fa3aa8f6649abab23615b546016d,2a578595fe351e7fce057e048c409b18,3432ed3d4466fac2f2ead83ab354e333,6.409801e+10,34.296596,-119.296596,Ventura,CA,urban,Ventura Unif School District,...,Literature & Writing,Literacy & Language,Books,highest poverty,Grades 3-5,282.47,28.0,t,4/7/12,4/18/12
2,000134f07d4b30140d63262c871748ff,26bd60377bdbffb53a644a16c5308e82,dc8dcb501c3b2bb0b10e9c6ee2cd8afd,6.227100e+10,34.078625,-118.257834,Los Angeles,CA,urban,Los Angeles Unif Sch Dist,...,Social Sciences,History & Civics,Technology,high poverty,Grades 3-5,1012.38,56.0,f,1/30/12,4/15/12
3,0001f2d0b3827bba67cdbeaa248b832d,15d900805d9d716c051c671827109f45,8bea7e8c6e4279fca6276128db89292e,3.600090e+11,40.687286,-73.988217,Brooklyn,NY,urban,New York City Dept Of Ed,...,NaN,NaN,Books,high poverty,Grades PreK-2,175.33,23.0,f,10/11/12,12/5/12
4,0004536db996ba697ca72c9e058bfe69,400f8b82bb0143f6a40b217a517fe311,fbdefab6fe41e12c55886c610c110753,3.606870e+11,40.793018,-73.205635,Central Islip,NY,suburban,Central Islip Union Free SD,...,Literature & Writing,Literacy & Language,Technology,high poverty,Grades PreK-2,3591.11,150.0,f,1/8/13,3/25/13


In [68]:
df.columns

Index(['projectid', 'teacher_acctid', 'schoolid', 'school_ncesid',
       'school_latitude', 'school_longitude', 'school_city', 'school_state',
       'school_metro', 'school_district', 'school_county', 'school_charter',
       'school_magnet', 'teacher_prefix', 'primary_focus_subject',
       'primary_focus_area', 'secondary_focus_subject', 'secondary_focus_area',
       'resource_type', 'poverty_level', 'grade_level',
       'total_price_including_optional_support', 'students_reached',
       'eligible_double_your_impact_match', 'date_posted', 'datefullyfunded'],
      dtype='object')

## 2. Explore data

tbd

In [69]:
to_plot = ['school_state', 'school_metro', 'school_charter', 'school_magnet', 
           'teacher_prefix', 'primary_focus_subject', 'primary_focus_area',
           'secondary_focus_subject', 'secondary_focus_area',
           'resource_type', 'poverty_level', 'grade_level',
           'total_price_including_optional_support', 'students_reached',
           'eligible_double_your_impact_match']

# library.plot_distributions(df, varlist=to_plot)

In [70]:
numeric_features = ['total_price_including_optional_support', 'students_reached']

library.describe_data(df, varlist=numeric_features)

,total_price_including_optional_support,students_reached
count,124976.000000,124917.000000
mean,654.011811,95.445760
std,1098.015854,163.481912
min,92.000000,1.000000
25%,345.810000,23.000000
50%,510.500000,30.000000
75%,752.960000,100.000000
max,164382.840000,12143.000000


## 3. Pre-process data

Data cleaning steps:
1. Clean missing data
2. Make binary data into true binary

In [71]:
# 1. Clean missing data
df.isnull().sum(axis=0)

projectid                                     0
teacher_acctid                                0
schoolid                                      0
school_ncesid                              9233
school_latitude                               0
school_longitude                              0
school_city                                   0
school_state                                  0
school_metro                              15224
school_district                             172
school_county                                 0
school_charter                                0
school_magnet                                 0
teacher_prefix                                0
primary_focus_subject                        15
primary_focus_area                           15
secondary_focus_subject                   40556
secondary_focus_area                      40556
resource_type                                17
poverty_level                                 0
grade_level                             

In [72]:
# school_ncesid - 9233 missing, is an ID column, can drop
# school_metro - 15224 missing, categorical - replace with category for other schools sharing the same district?
# school_district - 172 missing, categorical - leave as is
# primary_focus_subject - 15 missing, categorical - leave as is
# primary_focus_area - 15 missing, categorical - leave as is
# secondary_focus_subject - 40556 missing, categorical - leave as is
# secondary_focus_area - 40556 missing, categorical - leave as is
# resource_type - 17 missing, categorical - leave as is
# grade_level - 3 missing, leave as is
# students_reached - 59 missing, integer, fill with mean

df['students_reached'] = library.fill_missing(df['students_reached'])

The `school_charter`, `school_magnet` and `eligible_double_your_impact_match` features are also binary variables coded as string variables, so we'll convert them to a true binary variable.

In [73]:
# 2. Make binary data into true binary
binary_features = ['school_charter', 'school_magnet', 'eligible_double_your_impact_match']

for i in binary_features:
    df[i] = np.where(df[i] == 't', 1, 0)

df.head()

,projectid,teacher_acctid,schoolid,school_ncesid,school_latitude,school_longitude,school_city,school_state,school_metro,school_district,...,secondary_focus_subject,secondary_focus_area,resource_type,poverty_level,grade_level,total_price_including_optional_support,students_reached,eligible_double_your_impact_match,date_posted,datefullyfunded
0,00001ccc0e81598c4bd86bacb94d7acb,96963218e74e10c3764a5cfb153e6fea,9f3f9f2c2da7edda5648ccd10554ed8c,1.709930e+11,41.807654,-87.673257,Chicago,IL,urban,Pershing Elem Network,...,Visual Arts,Music & The Arts,Supplies,highest poverty,Grades PreK-2,1498.61,31.0,0,4/14/13,5/2/13
1,0000fa3aa8f6649abab23615b546016d,2a578595fe351e7fce057e048c409b18,3432ed3d4466fac2f2ead83ab354e333,6.409801e+10,34.296596,-119.296596,Ventura,CA,urban,Ventura Unif School District,...,Literature & Writing,Literacy & Language,Books,highest poverty,Grades 3-5,282.47,28.0,1,4/7/12,4/18/12
2,000134f07d4b30140d63262c871748ff,26bd60377bdbffb53a644a16c5308e82,dc8dcb501c3b2bb0b10e9c6ee2cd8afd,6.227100e+10,34.078625,-118.257834,Los Angeles,CA,urban,Los Angeles Unif Sch Dist,...,Social Sciences,History & Civics,Technology,high poverty,Grades 3-5,1012.38,56.0,0,1/30/12,4/15/12
3,0001f2d0b3827bba67cdbeaa248b832d,15d900805d9d716c051c671827109f45,8bea7e8c6e4279fca6276128db89292e,3.600090e+11,40.687286,-73.988217,Brooklyn,NY,urban,New York City Dept Of Ed,...,NaN,NaN,Books,high poverty,Grades PreK-2,175.33,23.0,0,10/11/12,12/5/12
4,0004536db996ba697ca72c9e058bfe69,400f8b82bb0143f6a40b217a517fe311,fbdefab6fe41e12c55886c610c110753,3.606870e+11,40.793018,-73.205635,Central Islip,NY,suburban,Central Islip Union Free SD,...,Literature & Writing,Literacy & Language,Technology,high poverty,Grades PreK-2,3591.11,150.0,0,1/8/13,3/25/13


## 4. Generate features/predictors

1. Define label: funded_in_60_days
2. Make categorical data into dummies
3. Drop features that are too collinear

In [74]:
df['date_posted'] = pd.to_datetime(df['date_posted'], format='%m/%d/%y')
df['datefullyfunded'] = pd.to_datetime(df['datefullyfunded'], format='%m/%d/%y')
df['fully_funded_in_60_days'] = \
    (df['datefullyfunded'] - df['date_posted'] <= pd.to_timedelta(60, unit='days')).astype('int')
df.head()

,projectid,teacher_acctid,schoolid,school_ncesid,school_latitude,school_longitude,school_city,school_state,school_metro,school_district,...,secondary_focus_area,resource_type,poverty_level,grade_level,total_price_including_optional_support,students_reached,eligible_double_your_impact_match,date_posted,datefullyfunded,fully_funded_in_60_days
0,00001ccc0e81598c4bd86bacb94d7acb,96963218e74e10c3764a5cfb153e6fea,9f3f9f2c2da7edda5648ccd10554ed8c,1.709930e+11,41.807654,-87.673257,Chicago,IL,urban,Pershing Elem Network,...,Music & The Arts,Supplies,highest poverty,Grades PreK-2,1498.61,31.0,0,2013-04-14,2013-05-02,1
1,0000fa3aa8f6649abab23615b546016d,2a578595fe351e7fce057e048c409b18,3432ed3d4466fac2f2ead83ab354e333,6.409801e+10,34.296596,-119.296596,Ventura,CA,urban,Ventura Unif School District,...,Literacy & Language,Books,highest poverty,Grades 3-5,282.47,28.0,1,2012-04-07,2012-04-18,1
2,000134f07d4b30140d63262c871748ff,26bd60377bdbffb53a644a16c5308e82,dc8dcb501c3b2bb0b10e9c6ee2cd8afd,6.227100e+10,34.078625,-118.257834,Los Angeles,CA,urban,Los Angeles Unif Sch Dist,...,History & Civics,Technology,high poverty,Grades 3-5,1012.38,56.0,0,2012-01-30,2012-04-15,0
3,0001f2d0b3827bba67cdbeaa248b832d,15d900805d9d716c051c671827109f45,8bea7e8c6e4279fca6276128db89292e,3.600090e+11,40.687286,-73.988217,Brooklyn,NY,urban,New York City Dept Of Ed,...,NaN,Books,high poverty,Grades PreK-2,175.33,23.0,0,2012-10-11,2012-12-05,1
4,0004536db996ba697ca72c9e058bfe69,400f8b82bb0143f6a40b217a517fe311,fbdefab6fe41e12c55886c610c110753,3.606870e+11,40.793018,-73.205635,Central Islip,NY,suburban,Central Islip Union Free SD,...,Literacy & Language,Technology,high poverty,Grades PreK-2,3591.11,150.0,0,2013-01-08,2013-03-25,0


In [75]:
# 2. Make categorical data into dummies

# CATEGORICAL FEATURES 
# omit school_city, school_district, school_county
categorical_features = ['school_state', 'school_metro', 'teacher_prefix', 'resource_type', \
                        'primary_focus_subject', 'primary_focus_area', \
                        'secondary_focus_subject', 'secondary_focus_area']

for i in categorical_features:
    df = library.make_dummy_vars(df, i)
    
df = df.drop(labels=['school_city', 'school_district', 'school_county'], axis=1)

# ORDINAL FEATURES
ordinal_features = ['poverty_level', 'grade_level']

for i in ordinal_features:
    df = library.make_dummy_vars(df, i)

In [76]:
df.shape

(124976, 154)

In [77]:
df.columns

Index(['projectid', 'teacher_acctid', 'schoolid', 'school_ncesid',
       'school_latitude', 'school_longitude', 'school_charter',
       'school_magnet', 'total_price_including_optional_support',
       'students_reached',
       ...
       'secondary_focus_area_Music & The Arts',
       'secondary_focus_area_Special Needs', 'poverty_level_high poverty',
       'poverty_level_highest poverty', 'poverty_level_low poverty',
       'poverty_level_moderate poverty', 'grade_level_Grades 3-5',
       'grade_level_Grades 6-8', 'grade_level_Grades 9-12',
       'grade_level_Grades PreK-2'],
      dtype='object', length=154)

In [78]:
# 3. Drop features that are too collinear

# library.get_correlations(df, plot=True)

In [79]:
# Final data to feed into classifier
df.head()

,projectid,teacher_acctid,schoolid,school_ncesid,school_latitude,school_longitude,school_charter,school_magnet,total_price_including_optional_support,students_reached,...,secondary_focus_area_Music & The Arts,secondary_focus_area_Special Needs,poverty_level_high poverty,poverty_level_highest poverty,poverty_level_low poverty,poverty_level_moderate poverty,grade_level_Grades 3-5,grade_level_Grades 6-8,grade_level_Grades 9-12,grade_level_Grades PreK-2
0,00001ccc0e81598c4bd86bacb94d7acb,96963218e74e10c3764a5cfb153e6fea,9f3f9f2c2da7edda5648ccd10554ed8c,1.709930e+11,41.807654,-87.673257,0,0,1498.61,31.0,...,1,0,0,1,0,0,0,0,0,1
1,0000fa3aa8f6649abab23615b546016d,2a578595fe351e7fce057e048c409b18,3432ed3d4466fac2f2ead83ab354e333,6.409801e+10,34.296596,-119.296596,0,0,282.47,28.0,...,0,0,0,1,0,0,1,0,0,0
2,000134f07d4b30140d63262c871748ff,26bd60377bdbffb53a644a16c5308e82,dc8dcb501c3b2bb0b10e9c6ee2cd8afd,6.227100e+10,34.078625,-118.257834,0,0,1012.38,56.0,...,0,0,1,0,0,0,1,0,0,0
3,0001f2d0b3827bba67cdbeaa248b832d,15d900805d9d716c051c671827109f45,8bea7e8c6e4279fca6276128db89292e,3.600090e+11,40.687286,-73.988217,0,1,175.33,23.0,...,0,0,1,0,0,0,0,0,0,1
4,0004536db996ba697ca72c9e058bfe69,400f8b82bb0143f6a40b217a517fe311,fbdefab6fe41e12c55886c610c110753,3.606870e+11,40.793018,-73.205635,0,0,3591.11,150.0,...,0,0,1,0,0,0,0,0,0,1


## 5. Build classifiers / 6. Evalute classifiers

Model building and validation steps:
1. Split data into test and train sets, dropping extraneous ID columns
2. Train series of classifiers on training data
3. Evaluate accuracy of classifiers on test data

In [80]:
# Split data into test and train sets, dropping extraneous ID columns
df = df.drop(labels=config.ID_COLS, axis=1)
df = df.drop(labels=['date_posted', 'datefullyfunded'], axis=1)
df.head()

,school_latitude,school_longitude,school_charter,school_magnet,total_price_including_optional_support,students_reached,eligible_double_your_impact_match,fully_funded_in_60_days,school_state_AK,school_state_AL,...,secondary_focus_area_Music & The Arts,secondary_focus_area_Special Needs,poverty_level_high poverty,poverty_level_highest poverty,poverty_level_low poverty,poverty_level_moderate poverty,grade_level_Grades 3-5,grade_level_Grades 6-8,grade_level_Grades 9-12,grade_level_Grades PreK-2
0,41.807654,-87.673257,0,0,1498.61,31.0,0,1,0,0,...,1,0,0,1,0,0,0,0,0,1
1,34.296596,-119.296596,0,0,282.47,28.0,1,1,0,0,...,0,0,0,1,0,0,1,0,0,0
2,34.078625,-118.257834,0,0,1012.38,56.0,0,0,0,0,...,0,0,1,0,0,0,1,0,0,0
3,40.687286,-73.988217,0,1,175.33,23.0,0,1,0,0,...,0,0,1,0,0,0,0,0,0,1
4,40.793018,-73.205635,0,0,3591.11,150.0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1


In [81]:
x_train, x_test, y_train, y_test = library.split_data(df, label=config.LABEL)

In [87]:
# Set classifiers, parameters, precision-recall thresholds to loop over
classifiers = ['LogisticRegression', 'KNeighborsClassifier', 'DecisionTreeClassifier',
              'LinearSVC', 'RandomForestClassifier', 'AdaBoostClassifier', 
               'BaggingClassifier']

pr_thresholds = [0.01, 0.02, 0.05, 0.1, 0.2, 0.3, 0.5, 1]

# results_df = pd.DataFrame(columns=['classifier', 'at_k', 'accuracy', 'precision', 'recall', 'f1'])
results_df = pd.DataFrame()

for i in classifiers:
    # train model
    trained = library.train_classifier(x_train, y_train, 
                                       method=i, 
                                       param_dict=config.MODEL_PARAMS)

    # evalute results
    results_dict = library.validate_classifier(x_test, y_test, trained,
                                              pr_threshold=pr_thresholds)

    # save to dict
    results_df = results_df.append(results_dict, ignore_index=True)

results_df.set_index('classifier').reset_index()

Training LogisticRegression with params {'penalty': 'l1', 'solver': 'liblinear', 'random_state': 0}
Training KNeighborsClassifier with params {'weights': 'distance'}
Training DecisionTreeClassifier with params {'criterion': 'entropy', 'max_features': 'auto', 'random_state': 0}
Training LinearSVC with params {'penalty': 'l1', 'dual': False, 'C': 1.0, 'random_state': 0}


/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Training RandomForestClassifier with params {'criterion': 'entropy', 'max_features': 'auto', 'random_state': 0}
Training AdaBoostClassifier with params {'random_state': 0}
Training BaggingClassifier with params {'random_state': 0}


,classifier,accuracy,auc-roc,f1,precision_0.01,precision_0.02,precision_0.05,precision_0.1,precision_0.2,precision_0.3,precision_0.5,precision_1,recall_0.01,recall_0.02,recall_0.05,recall_0.1,recall_0.2,recall_0.3,recall_0.5,recall_1
0,LogisticRegression,0.714827,0.657564,0.830377,0.930667,0.922667,0.913600,0.898106,0.863182,0.842461,0.807212,0.720885,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.979087
1,KNeighborsClassifier,0.669565,0.594831,0.782767,0.853333,0.836000,0.817067,0.811683,0.809308,0.792052,0.769658,0.736642,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.835054
2,DecisionTreeClassifier,0.640146,0.562304,0.746990,0.757333,0.750667,0.750400,0.746599,0.746233,0.746177,0.747146,0.748872,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.745118
3,LinearSVC,0.712960,0.644777,0.832420,0.925333,0.909333,0.898667,0.881302,0.853847,0.831615,0.799637,0.712963,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.999963
4,RandomForestClassifier,0.709386,0.659073,0.817591,0.882667,0.896000,0.885867,0.886370,0.861048,0.846284,0.809666,0.739880,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.913543
5,AdaBoostClassifier,0.718721,0.692349,0.828202,0.965333,0.962667,0.956267,0.930648,0.901720,0.874022,0.827963,0.733495,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.950991
6,BaggingClassifier,0.713360,0.664267,0.817622,0.874667,0.888000,0.886933,0.883169,0.862115,0.846550,0.808973,0.748206,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.901235


to do:
1. incorporate temporal validation splits
2. subset feature sets?
3. tune individual model parameters